In [ ]:
# AI Workflow Generation with ADK Provider

This notebook demonstrates how to use AI providers (like Together AI) to automatically generate workflows from natural language descriptions.

## Features Covered

- Setting up the ADK provider with Together AI
- Generating workflows from natural language descriptions  
- Executing AI-generated workflows
- Best practices for prompt engineering

## Prerequisites

1. Install the SDK with ADK support: `pip install kubiya-workflow-sdk[adk]`
2. Set your API keys in .env:
   - `KUBIYA_API_KEY="your-kubiya-key"`
   - `TOGETHER_API_KEY="your-together-key"`


# AI Workflow Generation with ADK Provider

This notebook demonstrates how to use AI providers to automatically generate workflows from natural language descriptions. The ADK (Agent Development Kit) provider supports multiple LLM backends and can generate complex, production-ready workflows.

## Features Covered

- Setting up the ADK provider with different LLM backends (OpenAI, Anthropic, Google)
- Generating workflows from natural language descriptions
- Validating and refining generated workflows
- Compiling workflows for execution
- Advanced use cases with custom tools and multi-step workflows

## Prerequisites

1. Install the SDK with ADK support: `pip install kubiya-workflow-sdk[adk]`
2. Set your API keys:
   - `export KUBIYA_API_KEY="your-kubiya-key"`
   - `export OPENAI_API_KEY="your-openai-key"` (or other LLM provider keys)

In [ ]:
import os
import json
from dotenv import load_dotenv
from kubiya_workflow_sdk import workflow, execute_workflow_logged
from kubiya_workflow_sdk.execution import LogLevel
from kubiya_workflow_sdk.client import KubiyaClient

# Load environment variables
load_dotenv()

# Setup API keys
kubiya_api_key = os.getenv("KUBIYA_API_KEY")
together_api_key = os.getenv("TOGETHER_API_KEY")

if not kubiya_api_key:
    print("❌ KUBIYA_API_KEY not set")
    raise ValueError("KUBIYA_API_KEY is required")

if not together_api_key:
    print("⚠️  TOGETHER_API_KEY not set")
    print("   AI generation features will be limited")

print("✅ Kubiya API key configured")
print(f"✅ Together AI key configured (length: {len(together_api_key) if together_api_key else 0})")

# Initialize client
client = KubiyaClient(api_key=kubiya_api_key)
print("\n✅ Kubiya client initialized")

In [ ]:
# Try to use ADK provider with Together AI if available
try:
    from kubiya_workflow_sdk.providers import get_provider
    from kubiya_workflow_sdk.providers.adk import ADKConfig, ModelProvider
    
    if together_api_key:
        print("🤖 Setting up ADK provider with Together AI...")
        
        # Configure ADK to use Together AI
        config = ADKConfig(
            model_provider=ModelProvider.TOGETHER,
            together_api_key=together_api_key,
            orchestrator_model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
            generator_model="meta-llama/Llama-3.3-70B-Instruct-Turbo"
        )
        
        # Create provider
        provider = get_provider("adk", client=client, config=config)
        print("✅ ADK provider configured with Together AI")
        
        # Example: Generate a workflow
        prompt = "Create a workflow that monitors system health every 5 minutes and sends alerts if CPU > 80%"
        print(f"\n📝 Prompt: {prompt}")
        
        try:
            generated_workflow = provider.generate_workflow(prompt)
            print("\n🎯 Generated Workflow:")
            print(json.dumps(generated_workflow, indent=2))
        except Exception as e:
            print(f"\n⚠️  Could not generate workflow: {str(e)}")
            print("   Using example workflow instead")
            
except ImportError:
    print("ℹ️  ADK provider not available")
    print("   Install with: pip install kubiya-workflow-sdk[adk]")
except Exception as e:
    print(f"⚠️  Error setting up ADK: {str(e)}")


In [ ]:
# Example: Using ADK provider to generate workflows (requires additional setup)
# For now, let's demonstrate what an AI-generated workflow would look like

# If you have ADK configured with an AI provider, you can use:
# from kubiya_workflow_sdk.providers import get_provider
# provider = get_provider("adk", client=client, api_key=openai_key)
# workflow_def = provider.generate_workflow("Create a database backup workflow")

# Example of what an AI might generate from the prompt:
# "Create a workflow that backs up a MySQL database and uploads to S3"
ai_generated_workflow = (
    workflow("mysql-backup-to-s3")
    .description("AI-generated workflow to backup MySQL database and upload to S3")
    .params(
        db_host="${DB_HOST:-localhost}",
        db_name="${DB_NAME:-myapp}",
        db_user="${DB_USER:-root}",
        s3_bucket="${S3_BUCKET:-my-backups}",
        retention_days="${RETENTION_DAYS:-7}"
    )
    .step("create-backup-dir", "mkdir -p /tmp/backups")
    .step("backup-database", """
        mysqldump -h ${db_host} -u ${db_user} -p${DB_PASSWORD} ${db_name} \
        > /tmp/backups/${db_name}-$(date +%Y%m%d-%H%M%S).sql
    """)
    .step("compress-backup", "gzip /tmp/backups/*.sql")
    .step("upload-to-s3", """
        aws s3 cp /tmp/backups/ s3://${s3_bucket}/mysql-backups/ \
        --recursive --exclude '*' --include '*.gz'
    """)
    .step("cleanup-old-backups", """
        aws s3 ls s3://${s3_bucket}/mysql-backups/ | while read -r line; do
            createDate=$(echo $line | awk '{print $1" "$2}')
            createDate=$(date -d "$createDate" +%s)
            olderThan=$(date -d "${retention_days} days ago" +%s)
            if [[ $createDate -lt $olderThan ]]; then
                fileName=$(echo $line | awk '{print $4}')
                aws s3 rm s3://${s3_bucket}/mysql-backups/$fileName
            fi
        done
    """)
    .step("cleanup-local", "rm -rf /tmp/backups")
)

print("🤖 Example AI-Generated Workflow:")
print(json.dumps(ai_generated_workflow.to_dict(), indent=2))
print("\n💡 To use real AI generation, configure ADK with your preferred LLM provider")